# The Discrete (Log) Likelihood Class

The likelihood class is very similar to the prior class in that it is a nice container for (log) likelihood functions. In the case of the CTA there are 3 so called Instrument Response Functions that describe how the telescope responds to input data. Two of the three describe the noise in the measurements themselves, in this notebook we detail how one uses these functions within the package `GammaBayes`. 

There are two ways that the likelihoods can be used within this formalism. One can use either or with care both within calculations. The first method is where we treat each likelihood separately. We will first go through the general inputs to the `discrete_loglikelihood` class within `GammaBayes`.

In [1]:
import sys
sys.path.append('..')

from gammabayes.likelihood import discrete_loglikelihood

## Inputs

1. `name='[None]'`
2. `inputunit=None` 
3. `logfunction=None`
4. `axes=None`
5. `dependent_axes=None`
6. `axes_names='[None]'` 
7. `dependent_axes_names='[None]'`
8. `logjacob = 0`

The `name` parameter is a string input for the name of the likelihood your are instantiating, default is '[None]'
___

The `inputunit` is another chance to detail your function, it is a string detailing the units of the normalisation axes. For example, the `energy_dispersion` is a likelihood normalised over the reconstructed energy values of the CTA, so the value within the class would be something like, `inputunit= ['TeV']`.

___

The `logfunction` input is the backbone of the class and is generally a wrapper function for the CTA IRFs that doesn't take in values with units. To allow for as much variability as possible we have currently chosen to accept functions that don't require input units to allow __any__ applicable function to be used with ease. 

___

`axes` represents a tuple of the axes over which the likelihoods are normalised, for the case of the energy dispersion this would be something like the axis of possible energies that the CTA could measure. As stated in the `log prior class` tutorial if there is only one axis please format the input as `(axis1,)` not `(axis1)` as when looking at the number of axes python will say `1` for the first one and the number of elements within `axis1` for the second.
___

`dependent_axes` should contain the axes of which the likelihoods are dependent on but not normalised with respect to. For example, the energy dispersion IRF for the CTA is normalised with respect to the reconstructed or measured energy values of the CTA, but is dependent on the 'true energy' and 'true sky position' of the measurements and different input values of these parameters creates different probability distribution shapes. 

___

`axes_names` and `dependent_axes_names` are another set of 'niceities' that would allow someone to more easily figure out what the likelihoods are dependent on.

___

`logjacob` is natural log of the jacobian that would be used within normalisation for example. If the shapes of the `axes` are `(m_1,)`, `(m_2,)`, ..., `(m_n,)` then the shape of the `logjacob` must be `(m_1, m_2, ..., m_n)`.

## Separate (Log) Likelihoods

In the case of separate likelihood treatments slightly less computational cost is required on the end of simulations as the `energy dispersion` which describes how the CTA will reconstruct the energy of the gamma ray events it observes. The other IRF, the `point spread function`, is generally more computationally expensive due to it's higher demensionality.

We will first construct the likelihood for the energy dispersion.

### Energy Dispersion

Within the `utils` module in `GammaBayes` there is the IRF function wrapper `edisp_test` function which outputs the log of the `energy dispersion` probability values. The energy dispersion is normalised with respect to the energy axis of the CTA measurements but because we use the log $_{10}$ energy values we will require a log jacobian $\propto$ log $_{e}$ energy. The energy dispersion is then also depends on the true energy and sky position of gamma ray events detected by the CTA. For the discrete_loglikelihood we need to define the axes of possible values that these can take for the likelihood as was done in the `simulations` tutorial notebook.

So we first define these axes, with 
- measured energy going from 0.1 TeV to 100 TeV with 50 bins per decade,
- true energy going from 0.1 TeV to 100 TeV with 100 bins per decade,
- true longitude (first coord of sky position) going from -3.5 to 3.5 deg in galactic longitude with a resolution of 0.2
- true latitude (first coord of sky position) going from -3.0 to 3.0 deg in galactic latitude with a resolution of 0.2

In [2]:
from gammabayes.utils.utils import edisp_test

import numpy as np


# log10 energy axis
energy_bins_per_decade          = 50
log10_e_lowerbound          = np.log10(0.1)
log10_e_upperbound          = np.log10(100)
log10_e_range               = log10_e_upperbound-log10_e_lowerbound

log10_eaxis            = np.linspace(log10_e_lowerbound,log10_e_upperbound,int(np.round(log10_e_range*energy_bins_per_decade))+1)

# log10 true energy axis
true_energy_bins_per_decade      = 100
log10_e_true_lowerbound          = np.log10(0.1)
log10_e_true_upperbound          = np.log10(100)
log10_e_true_range               = log10_e_true_upperbound-log10_e_true_lowerbound

log10_eaxis_true            = np.linspace(log10_e_true_lowerbound,log10_e_true_upperbound,int(np.round(log10_e_true_range*true_energy_bins_per_decade))+1)

log10_mass = np.log10(1)


# spatial axes

true_value_spatial_resolution = 0.2

## longitude axis
longitude_true_axis_lowerbound   = -3.5
longitude_true_axis_upperbound   = 3.5
longitude_true_axis_range        = longitude_true_axis_upperbound-longitude_true_axis_lowerbound

longitude_axis_true           = np.linspace(longitude_true_axis_lowerbound, 
                                            longitude_true_axis_upperbound, 
                                            int(round(longitude_true_axis_range/true_value_spatial_resolution)+1)) 

## latitude axis
latitude_true_axis_lowerbound    = -3.
latitude_true_axis_upperbound    = 3.
latitude_true_axis_range         = latitude_true_axis_upperbound-latitude_true_axis_lowerbound

latitude_axis_true            = np.linspace(latitude_true_axis_lowerbound, 
                                            latitude_true_axis_upperbound, 
                                            int(round(latitude_true_axis_range/true_value_spatial_resolution)+1)) 



/Users/lpin0002/anaconda3/envs/testywesty/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


We can then put the `log10_eaxis` in the `axes` argument, and the `log10_eaxis_true`, `longitude_axis_true` and `latitude_axis_true` in the `dependent_axes` argument. With the logjacobian for the likelihood generated using the `makelogjacob` function in the `utils` module of `GammaBayes`.

In [3]:
from gammabayes.utils.utils import makelogjacob

edisp_like = discrete_loglikelihood(logfunction=edisp_test, 
                                    axes=(log10_eaxis,), axes_names='log10E recon',
                                    name='energy dispersion',
                                    dependent_axes=(log10_eaxis_true, longitude_axis_true, latitude_axis_true,), logjacob=makelogjacob(log10_eaxis),
                                    dependent_axes_names = ['log10E true', 'lon', 'lat'])
edisp_like

Number of input dimensions 1
boop
Axes shape: (151,)


discrete log likelihood class
---------------------------------
name = energy dispersion
logfunction type is <function edisp_test at 0x153183ec0>
input units of None
over axes log10E recon
with dependent axes ['log10E true', 'lon', 'lat']

The primary goal of this class is to be a useful container for the likelihood, within it being two main functions:
1. `sample`, for a set of dependent values, one can sample the resultant probability distribution on the axes values
2. `())`, an instance of the class to be used as a function (e.g. `edisp_like(axes_vals, dependent_axes_vals) = probability`)

Examples of this class in use can be found in the `simulations` tutorial notebook.

### Point Spread Function

The other IRF that describes the noise in CTA measurements is the 'Point Spread Function' which describes how the CTA reconstructs sky position. It is normalised with respect to the values of __measured__ sky position and like the energy dispersion depedent on the true values of energy and sky position for a gamma ray event detected by the CTA. So the normalisation axes are `longitude` and `latitude` and then we can re-use the dependent axes from the previous construction.

We define these two axes with the same bounds as the true axes above, but half the resolution.

In [4]:
# spatial axes

spatial_resolution = 0.4

## longitude axis
longitude_axis_lowerbound   = -3.5
longitude_axis_upperbound   = 3.5
longitude_axis_range        = longitude_axis_upperbound-longitude_axis_lowerbound

longitude_axis           = np.linspace(longitude_axis_lowerbound, 
                                            longitude_axis_upperbound, 
                                            int(round(longitude_axis_range/spatial_resolution)+1)) 

## latitude axis
latitude_axis_lowerbound    = -3.
latitude_axis_upperbound    = 3.
latitude_axis_range         = latitude_axis_upperbound-latitude_axis_lowerbound

latitude_axis            = np.linspace(latitude_axis_lowerbound, 
                                            latitude_axis_upperbound, 
                                            int(round(latitude_axis_range/spatial_resolution)+1)) 

The log-likelihood function we use is the `psf_test` function available through the `GammaBayes` `utils` module.

In [5]:
from gammabayes.utils.utils import psf_test

psf_like = discrete_loglikelihood(logfunction=psf_test, 
                                    axes=(longitude_axis, latitude_axis), axes_names=['longitude recon', 'latitude recon'],
                                    name='point spread function ',
                                    dependent_axes=(log10_eaxis_true, longitude_axis_true, latitude_axis_true,),
                                    dependent_axes_names = ['log10E true', 'lon', 'lat'])
psf_like

Number of input dimensions 2
beeeep
Number of data dimensions 2
Axes shape: (19, 16)


discrete log likelihood class
---------------------------------
name = point spread function 
logfunction type is <function psf_test at 0x1535e1800>
input units of None
over axes ['longitude recon', 'latitude recon']
with dependent axes ['log10E true', 'lon', 'lat']

## One Likelihood

At the beginning of the tutorial we mentioned that there were two main ways to imeplement likelihoods within the Bayesian framework implemented by `GammaBayes`, the second way is to have a single likelihood that combines both of the IRFs/likelihoods above.

This means the `axes` argument is all the axes for the IRFs above, with all the same dependent axes. For the CTA, this translates into 6 dimensions, this uses even more memory than the PSF due to an extra dimension, so one must be careful when using this function. However, if one is just generating a matrix of probabilities of reconstructed values for a measured gamma ray event, the dimensionality of the result is only 3 so there is not such a demand for memory as there would be for 6.

In [8]:
from gammabayes.utils.utils import single_likelihood

single_likelihood_class_instance = discrete_loglikelihood(logfunction=single_likelihood, 
                                    axes=(log10_eaxis, longitude_axis, latitude_axis,), axes_names=['log10E','lon', 'lat'],
                                    name='full CTA likelihood',
                                    dependent_axes=(log10_eaxis_true, longitude_axis_true, latitude_axis_true,), logjacob=np.meshgrid(makelogjacob(log10_eaxis),longitude_axis, latitude_axis, indexing='ij')[0],
                                    dependent_axes_names = ['log10E true', 'lon true', 'lat true'])
single_likelihood_class_instance

Number of input dimensions 3
beeeep
Number of data dimensions 3
Axes shape: (151, 19, 16)


discrete log likelihood class
---------------------------------
name = full CTA likelihood
logfunction type is <function single_likelihood at 0x1535e18a0>
input units of None
over axes ['log10E', 'lon', 'lat']
with dependent axes ['log10E true', 'lon true', 'lat true']